In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import pandas as pd
import zipfile
import json
import requests
from tqdm import tqdm
import os
import errno
import re
import random
import numpy as np
import sys
import copy


import networkx as nx
#import matplotlib.pyplot as plt
#from ipywidgets import interact
#import ipywidgets as widgets
#from ipywidgets import Layout
#from IPython.display import display, clear_output

#Adjustment 1

You must download the network and utils into your own Google Drive.

In [13]:
#Set folder for remote drive
folder = '/content/drive/My Drive/tcma/'
#replace tcma with your file path

#importing utils for the method which downloads the current text json files
os.chdir(folder + 'network/utils/')
from utils import oracc_download

# This is a user defined module that searches through the texts to find the entities in the text that
# are people and places, to be imported as nodes into the network
os.chdir(folder + 'network/')
import rank_parser4 as rp

In [14]:
def parsejson(text):
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            #print('cdl in JSON')
            parsejson(JSONobject)
        if "label" in JSONobject:
            meta_d["label"] = JSONobject['label']
        if "f" in JSONobject:
            lemma = JSONobject["f"]
            if "ftype" in JSONobject:   # you don't need this - useful for distinguishing between regular text and year names 
                lemma['ftype'] = JSONobject['ftype']
            lemma["id_word"] = JSONobject["ref"]
            lemma['label'] = meta_d["label"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
            #print('Appending Lemma: ' + str(lemma))
        if "strict" in JSONobject and JSONobject["strict"] == "1":
            lemma = {key: JSONobject[key] for key in dollar_keys}
            lemma["id_word"] = JSONobject["ref"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
    return

In [15]:
os.chdir(folder + 'network/')

projects = [
      'epsd2-admin-ur3'

]
#projects = oracc_download(projects,'epsd2') #DOWNLOAD REDUNDANCY
projects

['epsd2-admin-ur3']

In [17]:
lemm_l = []
meta_d = {"label": None, "id_text": None}
dollar_keys = ["extent", "scope", "state"]

df_cat = pd.DataFrame()
used_pnums = []
cat_d = {}
for project in projects:
  print('Project: ' + str(project))
  # EDITED PROJECT REPLACE
  z = zipfile.ZipFile('jsonzip/epsd2-admin-ur3.zip')
  #print(file + " does not exist or is not a proper ZIP file")
  files = z.namelist()     # list of all the files in the ZIP
  files = [name for name in files if "corpusjson" in name and name[-5:] == '.json']
  #### EDITED THE CAT FILE READ 
  cat_file = z.read('epsd2/admin/ur3/catalogue.json').decode('utf-8')
  cat_json = json.loads(cat_file)
  cat_d.update(dict(cat_json['members']))
  #df_cat = pd.concat([df_cat,pd.DataFrame(cat_json['members']).T])                          #that holds all the P, Q, and X numbers.

  for filename in tqdm(files):       #iterate over the file names
      id_text = filename[-12:-5]
      if id_text in used_pnums:
        continue
      else:
        used_pnums.append(id_text)
      meta_d["id_text"] = id_text

      st = z.read(filename).decode('utf-8')         #read and decode the json file of one particular text
      data_json = json.loads(st)                # make it into a json object (essentially a dictionary)
      #print(str(data_json))
      parsejson(data_json)               # and send to the parsejson() function
  z.close()

df_cat = pd.DataFrame(cat_d).T
words_df = pd.DataFrame(lemm_l)
#words_df

Project: epsd2-admin-ur3


100%|██████████| 80181/80181 [11:34<00:00, 115.39it/s]


In [ ]:
words_df = words_df.fillna('')   # replace NaN (Not a Number) with empty string

findreplace = {' ' : '-', ',' : ''}
words_df = words_df.replace({'gw' : findreplace, 'sense' : findreplace}, regex=True)

words_df['id_line'] = [int(wordid.split('.')[1]) for wordid in words_df['id_word']]

words_df["norm1"] = words_df["norm"]
words_df.loc[words_df["norm1"] == "" , 'norm1'] = words_df['form']

words_df['lemma'] = words_df["cf"] + "[" + words_df["gw"] + "]" + words_df["pos"]
words_df.loc[words_df["cf"] == "" , 'lemma'] = words_df['form'] + "[NA]NA"
words_df.loc[words_df["form"] == "", 'lemma'] = ""

#words_df = words_df.merge(df_cat[['dossier_list']],how='left',left_on='id_text',right_index=True)

d = words_df.to_dict(orient='index')



Check to see if your dataframe is displaying correctly

In [ ]:
words_df